In [1]:
import os
import math
import collections

In [80]:
class books:
    """Return the maximum possible scores of books."""

    def __init__(self, filepath):
        """Initiate books, libraries, days, scores, libraBook."""
        with open(filepath, "r") as file:
            dat = file.readlines()
            self.books, self.libraries, self.days = [int(item) for item in dat[0][:-1].split(" ")]
            self.scores = [int(item) for item in dat[1][:-1].split(" ")]
            self.libraBook = []
            for i in range(2, len(dat), 2):
                if dat[i] == "\n":
                    break
                self.libraBook.append([[int(item) for item in dat[i][:-1].split(" ")],
                                       sorted([int(item) for item in dat[i + 1][:-1].split(" ")],
                                              key=lambda x: self.scores[x])])
                self.libraBook[-1][0].append(sum(self.scores[b] for b in self.libraBook[-1][1]))
            self.minSignup = min(x[0][1] for x in self.libraBook)
#         print(self.books, self.libraries, self.days, self.scores, self.libraBook)
        
    def current_score(self, lib_left: list, days_left: int, book_scanned: set):
        """
        Calculate score and trade-off score when selecting lib to sign up.
        """
        current_score, rest_days, trade_off_score = [], [], []
        
        book_left = set(range(self.books)) - set(book_scanned)
        mean_score = sum([self.scores[book] for book in book_left]) / len(book_left)
        
        for lib in lib_left:
            scores = [self.scores[book] for book in self.libraBook[lib][1] if book not in book_scanned]
            scores.sort(reverse=True)
            days_after_sign = max(0, days_left - self.libraBook[lib][0][1])
            scan_per_day = self.libraBook[lib][0][2]
            
            score = sum(scores[:days_after_sign * scan_per_day])
            current_score.append(score)
            
            rest_day = max(0, days_after_sign - math.ceil(len(scores) / scan_per_day))
            rest_days.append(rest_day)
            
            # trade_off_score = score - opportunity_cost
            trade_off_score.append(score - rest_day * mean_score)
            
        return current_score, rest_days, trade_off_score
    
    def scan_book(self, lib, days_left, book_scanned):
        
        days_after_sign = max(0, days_left - self.libraBook[lib][0][1])
        scan_per_day = self.libraBook[lib][0][2]

        book_list = [book for book in self.libraBook[lib][1] if book not in book_scanned]
        score_list = [self.scores[book] for book in book_list]
        score_dict = collections.OrderedDict(dict(zip(book_list, score_list)))
        sorted_book_list = list(score_dict.keys())

        scan_list = sorted_book_list[:days_after_sign * scan_per_day]

        return scan_list

        
    def MaxScore(self):
        
        days_left = self.days
        lib_left = list(range(self.libraries))
        book_scanned = set()
        
        lib_num = 0
        scan_process = []
        
        while days_left > 0 and len(book_scanned) < self.books:
            current_score, rest_days, trade_off_score = self.current_score(lib_left, days_left, book_scanned)
            next_lib = lib_left[trade_off_score.index(max(trade_off_score))]
            
            scan_order = self.scan_book(next_lib, days_left, book_scanned)
            scan_process += [[next_lib, len(scan_order)], scan_order]
            
            lib_num += 1
            days_left -= self.libraBook[next_lib][0][1]
            lib_left.remove(next_lib)
            book_scanned.update(scan_order)
            
        if scan_process[-1] == []:
            scan_process = scan_process[:-2]
            lib_num -= 1
            
        return [lib_num, *scan_process]
    
    def total_score(self):
        
        total_book = []
        result = self.MaxScore()
        lib_num = result[0]
        for i in range(1, lib_num + 1):
            total_book += result[i * 2]
            
        scores = [self.scores[book] for book in total_book]
        
        return sum(scores)
    

os.chdir(".")
for name in ["a_example"]:
    ans = books(name + ".txt")

In [83]:
books("a_example.txt").total_score(), books("b_read_on.txt").total_score()

(21, 5822900)

In [50]:
for name in ["a_example", "b_read_on", "c_incunabula", "d_tough_choices", "e_so_many_books", "f_libraries_of_the_world"]:
    ans = books(name + ".txt")
    print("{} total score is {}".format(name, ans.total_score()))

a_example total score is 21
b_read_on total score is 5822900
c_incunabula total score is 522315
d_tough_choices total score is 5028530
e_so_many_books total score is 897732
f_libraries_of_the_world total score is 1405258


In [84]:
for name in ["a_example", "b_read_on", "c_incunabula", "d_tough_choices", "e_so_many_books", "f_libraries_of_the_world"]:
    ans = books(name + ".txt").MaxScore()
    file = open(name + "_ans.txt", "w")
    file.write(str(ans[0]) + "\n")
    for a in ans[1:]:
        file.write(" ".join(str(A) for A in a) + "\n")
    file.close()